## Data Parallel Training for ThirdAI's UDT

This notebook shows how to run Data Parallel Training for ThirdAI's UDT. We will be using CLINC 150 small dataset for training and evaluation for this demo. But, you can easily replace this with your workload. 

ThirdAI's Distributed Data Parallel Training assumes that you already have a ray cluster running. For this demo, we would be using the ray mock cluster to simulate the ray cluster. For seetting up a ray cluster, see here: https://docs.ray.io/en/latest/cluster/getting-started.html

In [1]:
#!pip3 install thirdai --upgrade --no-cache-dir --force-reinstall
!pip3 install ray
!pip3 install torch

import thirdai
from thirdai import bolt
import thirdai.distributed_bolt as dist     

/bin/pip3:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point
/bin/pip3:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point


/home/pratik/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-18 08:41:27,838	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-08-18 08:41:29,151	WARNING collective.py:20 -- NCCL seems unavailable. Please install Cupy following the guide at: https://docs.cupy.dev/en/stable/install.html.
2023-08-18 08:41:29,315	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-08-18 08:41:29,573	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Ray Cluster Initialization
For the purpose of this demo, we will be initializing a mock ray cluster of 2 nodes here.

In [2]:
import ray
from ray.air import ScalingConfig, session

cpus_per_node = (dist.get_num_cpus() - 1) // 2

ray.init(ignore_reinit_error=True, runtime_env={"env_vars": {"OMP_NUM_THREADS": f"{cpus_per_node}"}})
scaling_config = ScalingConfig(
    num_workers=2,
    use_gpu=False,
    trainer_resources={"CPU": 1},
    resources_per_worker={"CPU": cpus_per_node},
    placement_strategy="PACK",
)
thirdai.licensing.activate("WUAT-V7FP-TXLJ-97KR-3MCV-H4UC-7ERL-JYAF") 

2023-08-18 08:41:32,187	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


In [3]:
import os
print(os.getenv('OMP_NUM_THREADS'))

None


# Dataset Download

We will use the demos module in the thirdai package to download the CLINC 150 small dataset. You can replace theis step and the next step with a download method and a UDT initialization that is specific to your dataset.

In [4]:
from thirdai.demos import download_clinc_dataset

train_filenames, test_filename , _ = download_clinc_dataset(num_training_files=2, clinc_small=True)

# UDT Initialization
We can now create a UDT model by passing in the types of each column in the dataset and the target column we want to be able to predict.

In [5]:
def get_udt_model():
    model = bolt.UniversalDeepTransformer(
        data_types={
            "text": bolt.types.text(),
            "category": bolt.types.categorical(),
        },
        target="category",
        n_target_classes=151,
        integer_target=True,
    )
    return model
    
def train_loop_per_worker(config):
    # thirdai.licensing.deactivate()
    thirdai.licensing.activate("WUAT-V7FP-TXLJ-97KR-3MCV-H4UC-7ERL-JYAF") 
    thirdai.logging.setup(log_to_stderr=False, path="log.txt", level="info")
    
    model = get_udt_model()
    model = dist.prepare_model(model)

    metrics = model.train_distributed_v2(
        filename=os.path.join(config["curr_dir"], train_filenames[session.get_world_rank()]),
        learning_rate=0.02,
        epochs=1,
        batch_size=256,
        metrics=["categorical_accuracy"],
        verbose=True,
    )

    session.report(
        metrics=metrics,
        checkpoint=dist.UDTCheckPoint.from_model(model),
    )



## Distributed Training

We will now train a UDT model in distributed data parallel fashion. Feel free to customize the number of epochs and the learning rate; we have chosen values that give good convergence. 

In [6]:
import os
from ray.train.torch import TorchConfig

trainer = dist.BoltTrainer(
    train_loop_per_worker=train_loop_per_worker,
    train_loop_config={
        "curr_dir": os.path.abspath(os.getcwd()),
    },
    scaling_config=scaling_config,
    backend_config=TorchConfig(backend="gloo"),
)

result_checkpoint_and_history = trainer.fit()


(pid=3006101) NCCL seems unavailable. Please install Cupy following the guide at: https://docs.cupy.dev/en/stable/install.html.
(BoltTrainer pid=3006101) Starting distributed worker processes: ['3006208 (192.168.1.10)', '3006209 (192.168.1.10)']
(RayTrainWorker pid=3006208) Setting up process group for: env:// [rank=0, world_size=2]


(RayTrainWorker pid=3006208) loaded data | source '/home/pratik/Demos/distributed/clinc_train_0.csv' | vectors 3750 | batches 15 | time 0s | complete
(RayTrainWorker pid=3006208) 
train: [                                                  ] 0%          
train: [===                                               ] 6%
train: [=======                                           ] 13%
train: [==========                                        ] 20%
train: [=============                                     ] 26%
train: [=================                                 ] 33%
train: [====================                              ] 40%
train: [=======================                           ] 46%
(RayTrainWorker pid=3006209) loaded data | source '/home/pratik/Demos/distributed/clinc_train_1.csv' | vectors 3750 | batches 15 | time 0s | complete
(RayTrainWorker pid=3006209) 
train: [===========================                       ] 53%
train: [==============================                  

2023-08-18 08:42:04,784	INFO tune.py:1148 -- Total run time: 28.28 seconds (28.24 seconds for the tuning loop).


# Evaluation
Evaluating the performance of the UDT model is just two line!

In [7]:
model = result_checkpoint_and_history.checkpoint.get_model()
model.evaluate(test_filename, metrics=["categorical_accuracy"])

loaded data | source './clinc_test.csv' | vectors 4500 | batches 3 | time 0s | complete

validate | epoch 0 | train_steps 15 | val_categorical_accuracy=0.830889  | val_batches 3 | time 0s



{'val_times': [0.0], 'val_categorical_accuracy': [0.8308888673782349]}

# Ray Cluster Teardown

In [8]:
ray.shutdown()